# Sentiment Classification

The Notebook presents a simple sentiment classification system using Keras LSTM network.
## Importing necessary libraries

In [1]:
import ast
import helper
import re
import string
import numpy as np
import pickle

## Reading data and creating pairs of sentence and sentiment

In [2]:
f = open("imdb_labelled.txt")
data1 = f.readlines()
f.close()

f = open("amazon_cells_labelled.txt")
data2 = f.readlines()
f.close()

f = open("yelp_labelled.txt")
data3 = f.readlines()
f.close()

pairs = []
for one in data1:
    pairs.append(one.strip("\n").lower().split("\t"))
    
for one in data2:
    pairs.append(one.strip("\n").lower().split("\t"))
    
for one in data3:
    pairs.append(one.strip("\n").lower().split("\t"))

## Loading pretrained word embeddings

In [3]:
f = open("glove.6B.50d.txt", encoding='utf-8')
emb_dict = dict()
for line in f:
    values = line.split()
    word = values[0]
    coef = values[1:]
    emb_dict[word] = coef
f.close()

## Cleaning text

In [4]:
translation = str.maketrans("","", string.punctuation)
new_pairs = []
for val in pairs:
    sent = val[0]
    sent = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-])\s*", r"\1 ", sent)
    sent = sent.replace('-', ' ')
    sent = sent.replace('/', ' ')
    sent = sent.translate(translation)
    new_pairs.append([sent, val[1]])

## Geting set of known words accordig to pretrained embeddings

In [5]:
vocab = set(['eos', 'pad'])
for pair in new_pairs:
    sent = pair[0]
    for word in sent.split():
        vocab.add(word) 

## Getting unknown words

In [6]:
unk = -1
unk_set = set()
for word in vocab:
    if emb_dict.get(word, -1) == -1:
        unk += 1
        unk_set.add(word)
print(unk)

208


## Replacing unknown words with ```unk``` token

In [7]:
n = []
for pair in new_pairs:
    sent = pair[0]
    sent = sent.split()
    for i, word in enumerate(sent):
        if(word in unk_set):
            sent[i] = "unk"
    new_sent = ' '.join(sent)
    n.append([new_sent, pair[1]])

## Appending ```eos``` token at end of sentencs

In [8]:
vocab = set(['eos', 'pad'])
for pair in n:
    sent = pair[0]
    for word in sent.split():
        vocab.add(word)   

## creating necessary dictionaries

In [9]:
word_to_index = dict({'pad' : 0})
print(word_to_index)
for word in vocab:
    if word != 'pad':
        word_to_index[word] = len(word_to_index)

index_to_word = dict(zip(word_to_index.values(), word_to_index.keys()))

{'pad': 0}


In [10]:
useful_emb = np.zeros((len(word_to_index), 50))
for word, index in word_to_index.items():
    useful_emb[index] = emb_dict[word]

## Creating indexed data and sentiment pair

In [11]:
indexed_pairs = []
for pair in n:
    sent = pair[0].split()
    index_sent = []
    for word in sent:
        index_sent.append(word_to_index[word])
    index_sent.append(word_to_index['eos'])
    indexed_pairs.append([index_sent, ast.literal_eval(pair[1])])

## Dumping important data structures

In [12]:
f = open("important_data.obj", "wb")
pickle.dump([vocab, word_to_index, useful_emb], f, pickle.HIGHEST_PROTOCOL)
f.close()
del(emb_dict)

## Spliting data in train and test set

In [13]:
split_point = int(len(indexed_pairs) * 0.2)
train_data = indexed_pairs[:-split_point]
valid_data = indexed_pairs[-split_point:]

## Creating batch generators for train and test sets

In [14]:
train = helper.batch_generator(train_data, 20)
valid = helper.batch_generator(valid_data, 20)

## Creating Model

In [15]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, Dropout

Using TensorFlow backend.


In [16]:
def base_model():
    m = Sequential()
    m.add(Embedding(len(word_to_index),50,weights=[useful_emb], input_shape=[None,]))
    m.add(LSTM(512, return_sequences = True))
    m.add(Dropout(0.5))
    m.add(LSTM(512))
    m.add(Dense(1, activation='sigmoid'))
    return m

In [17]:
model = base_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          251500    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 512)         1153024   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 3,504,237
Trainable params: 3,504,237
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [19]:
from keras.callbacks import ModelCheckpoint
num_epochs = 100
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
checkpointer = ModelCheckpoint(filepath='model-{epoch:02d}.hdf5', verbose=1)
model.fit_generator(train.batch(), len(train_data)//20, num_epochs, validation_data = valid.batch(),
                    validation_steps = len(valid_data)//20, callbacks = [checkpointer])

Epoch 1/100
120/120 [==============================] - 49s - loss: 0.7037 - acc: 0.5238 - val_loss: 0.7126 - val_acc: 0.4767
Epoch 2/100
120/120 [==============================] - 47s - loss: 0.6951 - acc: 0.4929 - val_loss: 0.7614 - val_acc: 0.5000
Epoch 3/100
120/120 [==============================] - 47s - loss: 0.6941 - acc: 0.5304 - val_loss: 0.7309 - val_acc: 0.4617
Epoch 4/100
120/120 [==============================] - 46s - loss: 0.6822 - acc: 0.5463 - val_loss: 0.7048 - val_acc: 0.4617
Epoch 5/100
120/120 [==============================] - 47s - loss: 0.6981 - acc: 0.5058 - val_loss: 0.6935 - val_acc: 0.4617
Epoch 6/100
120/120 [==============================] - 47s - loss: 0.6946 - acc: 0.4879 - val_loss: 0.6890 - val_acc: 0.4733
Epoch 7/100
120/120 [==============================] - 46s - loss: 0.6921 - acc: 0.5042 - val_loss: 0.6811 - val_acc: 0.4767
Epoch 8/100
120/120 [==============================] - 47s - loss: 0.6187 - acc: 0.6338 - val_loss: 0.5330 - val_acc: 0.7750


120/120 [==============================] - 47s - loss: 0.0610 - acc: 0.9850 - val_loss: 0.6509 - val_acc: 0.8400
Epoch 35/100
120/120 [==============================] - 47s - loss: 0.0315 - acc: 0.9925 - val_loss: 0.8575 - val_acc: 0.8467
Epoch 36/100
120/120 [==============================] - 47s - loss: 0.0271 - acc: 0.9946 - val_loss: 0.8924 - val_acc: 0.8433
Epoch 37/100
120/120 [==============================] - 47s - loss: 0.0259 - acc: 0.9950 - val_loss: 0.9897 - val_acc: 0.8417
Epoch 38/100
120/120 [==============================] - 49s - loss: 0.0240 - acc: 0.9950 - val_loss: 0.9385 - val_acc: 0.8433
Epoch 39/100
120/120 [==============================] - 48s - loss: 0.0219 - acc: 0.9954 - val_loss: 1.0885 - val_acc: 0.8333
Epoch 40/100
120/120 [==============================] - 47s - loss: 0.0240 - acc: 0.9954 - val_loss: 1.0754 - val_acc: 0.8383
Epoch 41/100
120/120 [==============================] - 47s - loss: 0.0213 - acc: 0.9958 - val_loss: 1.0825 - val_acc: 0.8483
Epoch

120/120 [==============================] - 47s - loss: 1.4827e-04 - acc: 1.0000 - val_loss: 1.4482 - val_acc: 0.8233
Epoch 67/100
120/120 [==============================] - 47s - loss: 9.4128e-05 - acc: 1.0000 - val_loss: 1.4799 - val_acc: 0.8233
Epoch 68/100
120/120 [==============================] - 48s - loss: 7.1284e-05 - acc: 1.0000 - val_loss: 1.5085 - val_acc: 0.8217
Epoch 69/100
120/120 [==============================] - 47s - loss: 5.7195e-05 - acc: 1.0000 - val_loss: 1.5339 - val_acc: 0.8217
Epoch 70/100
120/120 [==============================] - 47s - loss: 4.8333e-05 - acc: 1.0000 - val_loss: 1.5568 - val_acc: 0.8217
Epoch 71/100
120/120 [==============================] - 48s - loss: 4.0722e-05 - acc: 1.0000 - val_loss: 1.5781 - val_acc: 0.8233
Epoch 72/100
120/120 [==============================] - 47s - loss: 3.7488e-05 - acc: 1.0000 - val_loss: 1.5995 - val_acc: 0.8250
Epoch 73/100
120/120 [==============================] - 47s - loss: 2.8287e-05 - acc: 1.0000 - val_loss

120/120 [==============================] - 47s - loss: 0.0156 - acc: 0.9967 - val_loss: 1.1038 - val_acc: 0.8183
Epoch 99/100
120/120 [==============================] - 47s - loss: 0.0145 - acc: 0.9971 - val_loss: 1.0789 - val_acc: 0.8133
Epoch 100/100
120/120 [==============================] - 47s - loss: 0.0056 - acc: 0.9992 - val_loss: 1.0268 - val_acc: 0.8217


## Loading Trained model
The following section Loads the trained model and lets you predict the sentiment of your sentences.

In [20]:
from keras.models import load_model
import numpy as np
import re
import string
import pickle

f = open("important_data.obj", "rb")
vocab, word_to_index, useful_emb = pickle.load(f)
f.close()
m = load_model(r"model-99.hdf5")
translation = str.maketrans("","", string.punctuation)


In [22]:
while True:
    choice = int(input("Enter 0 to stop else Enter 1: "))
    if choice == 0:
        break
    sent = input("Enter your sentence: ")
    sent = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-])\s*", r"\1 ", sent).lower()
    sent = sent.replace('-', ' ')
    sent = sent.replace('/', ' ')
    sent = sent.translate(translation)
    sent = sent.split()
    for i, word in enumerate(sent):
        if(word not in vocab):
            sent[i] = "unk"

    index_sent = []
    for word in sent:
        index_sent.append(word_to_index[word])
    index_sent.append(word_to_index['eos'])
    index_sent = np.reshape(index_sent, (1, len(index_sent)))
    prediction = m.predict(index_sent)
    if prediction[0][0] >= 0.5:
        print("Sentiment: Positive")
    else:
        print("Sentiment: Negative")

Enter 0 to stop else Enter 1: 1
Enter your sentence: I don't like this.
Sentiment: Negative
Enter 0 to stop else Enter 1: 1
Enter your sentence: Just love it, EXCELLENT MOVIE!!!
Sentiment: Positive
Enter 0 to stop else Enter 1: 1
Enter your sentence: Shit movie.
Sentiment: Negative
Enter 0 to stop else Enter 1: 1
Enter your sentence: I don't recommend it to anyone at all.
Sentiment: Negative
Enter 0 to stop else Enter 1: 1
Enter your sentence: EXCELLENT BUY!!!
Sentiment: Positive
Enter 0 to stop else Enter 1: 0
